In [1]:
import numpy as np


# Perf stat measurements

Measuring package and dram power using `perf stat`, roughly `73.7W` when node is idling:

**Note: This is when kepler+prometheus is already running on kube-worker-68 node, I expect this to be lower when the system does not have these programs running in the background**

```
[peaks@kube-worker-68 ~]$ for ((i=0;i<3;i++)); do perf stat -a -e power/energy-pkg/,power/energy-ram/ sleep 60; done
 Performance counter stats for 'system wide':

          4,423.93 Joules power/energy-pkg/
          4,437.35 Joules power/energy-ram/

      60.006488329 seconds time elapsed
 Performance counter stats for 'system wide':

          4,434.37 Joules power/energy-pkg/
          4,465.97 Joules power/energy-ram/

      60.005678481 seconds time elapsed
 Performance counter stats for 'system wide':

          4,368.88 Joules power/energy-pkg/
          4,345.88 Joules power/energy-ram/

      60.004851317 seconds time elapsed
```

# Rapl log measurements

Similar numbers reported from the Rapl logging tool:
```
cat /tmp/rap.log
71.776428 70.815857
72.162109 72.975739
73.529907 73.922562
73.424866 72.284805
81.910828 75.892334
75.152527 73.884552
```

# Kepler measurements

These measurements are collected manually by `kubectl port-forward svc/kepler-exporter 9102:9102` then doing `curl localhost:9102/metrics`

In [14]:
for i in range(1,4):
    fs = f"kepler.sleep.S{i}"
    fe = f"kepler.sleep.E{i}"

    with open(fe) as f:
        lines = f.readlines()
    for line in lines:
        if "idle" in line and "kepler_container_package_joules_total" in line:
            jpe = float(line.split(' ')[1])
        if "idle" in line and "kepler_container_dram_joules_total" in line:
            jre = float(line.split(' ')[1])

    with open(fs) as f:
        lines = f.readlines()
    for line in lines:
        if "idle" in line and "kepler_container_package_joules_total" in line:
            jps = float(line.split(' ')[1])
        if "idle" in line and "kepler_container_dram_joules_total" in line:
            jrs = float(line.split(' ')[1])

    print(f"Run {i}:")
    print(f"\t kepler_container_package_joules_total: {jpe-jps}")
    print(f"\t kepler_container_dram_joules_total: {jre-jrs}\n")
    

Run 1:
	 kepler_container_package_joules_total: 106.08899999999998
	 kepler_container_dram_joules_total: 107.262

Run 2:
	 kepler_container_package_joules_total: 96.666
	 kepler_container_dram_joules_total: 97.74000000000001

Run 3:
	 kepler_container_package_joules_total: 107.214
	 kepler_container_dram_joules_total: 108.168

